In [ ]:
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path):
    return np.loadtxt(path, dtype=float, skiprows = 1)

def readFileFromDialog():
    return readFile(getFilePathFromDialog(dataPath))

In [ ]:
dataPath = getDirectoryPathFromDialog('')

In [ ]:
traveledData = readFileFromDialog()

In [ ]:
Fig, Host = plt.subplots(len(traveledData[0]),1,squeeze=False, figsize = (10,6*len(traveledData[0])), dpi = 100)
for i in range(len(traveledData[0])):
    Host[i,0].plot(traveledData[1:,i],label='y={}'.format(traveledData[0,i]))
    Host[i,0].legend()

In [ ]:
NumberOfASeries = readFileFromDialog()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(NumberOfASeries,'.')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].hist(NumberOfASeries,bins = 41, density = True)
Ax[0,0].set_xlabel(r'$N_A$')
Ax[0,0].set_ylabel('number of occurences')

In [ ]:
PotEnergySeries = readFileFromDialog()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(PotEnergySeries,'.')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].hist(PotEnergySeries,bins = 41, density = True)
Ax[0,0].set_xlabel(r'$N_A$')
Ax[0,0].set_ylabel('number of occurences')